In [598]:
import numpy as np
import pandas as pd

In [599]:
df = pd.read_csv('train.csv')
np.random.seed(0)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [600]:
missing_value = df.isnull().sum()
missing_value[70:82]

ScreenPorch         0
PoolArea            0
PoolQC           1453
Fence            1179
MiscFeature      1406
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
SaleCondition       0
SalePrice           0
dtype: int64

In [601]:
total = np.prod(df.shape)
missing = missing_value.sum()
percent_missing = missing/total*100
print(percent_missing)

6.620158971757145


In [602]:
columns_with_na_dropped = df.dropna(axis=1)
columns_with_na_dropped.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [603]:
lot_frontage = df['LotFrontage']
lot_frontage.fillna(lot_frontage.median(), inplace=True)


In [604]:
df[['GarageQual', 'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish']]=df[['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual']].fillna('None')
df[['GarageCond', 'PoolQC', 'Fence', 'MiscFeature']]  = df[['GarageCond', 'PoolQC', 'Fence', 'MiscFeature']].fillna('None')

In [605]:
df['MasVnrArea']=df['MasVnrArea'].fillna(0.0)

In [606]:
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

In [607]:
df.loc[df['GarageType'].isna(), 'GarageYrBlt']=0
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)

In [608]:
df.dtypes[0:10]

Id               int64
MSSubClass       int64
MSZoning        object
LotFrontage    float64
LotArea          int64
Street          object
Alley           object
LotShape        object
LandContour     object
Utilities       object
dtype: object

In [609]:
def find_outliers(df):
    outlier_indices = {}
    for col in df.select_dtypes(['float64', 'int64']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        higher_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > higher_bound)].index.tolist()
        if outliers:
            outlier_indices[col] = outliers
    return outlier_indices


In [610]:
out = find_outliers(df)
for column, indices in out.items():
    print(f"Column {column} has {len(indices)} outliers")

Column MSSubClass has 103 outliers
Column LotFrontage has 106 outliers
Column LotArea has 69 outliers
Column OverallQual has 2 outliers
Column OverallCond has 125 outliers
Column YearBuilt has 7 outliers
Column MasVnrArea has 98 outliers
Column BsmtFinSF1 has 7 outliers
Column BsmtFinSF2 has 167 outliers
Column BsmtUnfSF has 29 outliers
Column TotalBsmtSF has 61 outliers
Column 1stFlrSF has 20 outliers
Column 2ndFlrSF has 2 outliers
Column LowQualFinSF has 26 outliers
Column GrLivArea has 31 outliers
Column BsmtFullBath has 1 outliers
Column BsmtHalfBath has 82 outliers
Column BedroomAbvGr has 35 outliers
Column KitchenAbvGr has 68 outliers
Column TotRmsAbvGrd has 30 outliers
Column Fireplaces has 5 outliers
Column GarageYrBlt has 81 outliers
Column GarageCars has 5 outliers
Column GarageArea has 21 outliers
Column WoodDeckSF has 32 outliers
Column OpenPorchSF has 77 outliers
Column EnclosedPorch has 208 outliers
Column 3SsnPorch has 24 outliers
Column ScreenPorch has 116 outliers
Colu

In [611]:
df = pd.get_dummies(df, columns = ['MSZoning', 'Street', 'Alley', 'Utilities',
                                           'Neighborhood','LotConfig', 'Condition1', 
                                           'Condition2', 'BldgType', 'HouseStyle',
                                           'RoofStyle', 'RoofMatl', 'Exterior1st',
                                           'Exterior2nd', 'Heating', 'Electrical',
                                           'FireplaceQu', 'MiscFeature', 'SaleType',
                                           'SaleCondition'], drop_first=True)
print(df.head())

   Id  MSSubClass  LotFrontage  LotArea LotShape LandContour LandSlope  \
0   1          60         65.0     8450      Reg         Lvl       Gtl   
1   2          20         80.0     9600      Reg         Lvl       Gtl   
2   3          60         68.0    11250      IR1         Lvl       Gtl   
3   4          70         60.0     9550      IR1         Lvl       Gtl   
4   5          60         84.0    14260      IR1         Lvl       Gtl   

   OverallQual  OverallCond  YearBuilt  ...  SaleType_ConLI SaleType_ConLw  \
0            7            5       2003  ...           False          False   
1            6            8       1976  ...           False          False   
2            7            5       2001  ...           False          False   
3            7            5       1915  ...           False          False   
4            8            5       2000  ...           False          False   

   SaleType_New SaleType_Oth SaleType_WD SaleCondition_AdjLand  \
0         False     

In [612]:
cols = df.select_dtypes(include=['object']).columns
print(cols)

Index(['LotShape', 'LandContour', 'LandSlope', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir',
       'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence'],
      dtype='object')


In [613]:
#Ordinal Encoding
lotshape_mapping = {'Reg':0, 'IR1':1, 'IR2':2, 'IR3':3}
landContour_mapping = {'Lvl':0, 'Bnk':1, 'HLS':2, 'Low':3}
landslope_mapping = {'Gtl':0, 'Mod':1, "Sev":2}
quality_mapping = {'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5 }
bsmt_cond_mapping = {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}
bsmyexposure_mapping = {'None':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}
functional_mapping = {'Sal':0, 'Sev':1, 'Maj2':2, 'Maj1':3, 'Mod':4, 'Min2':5, 'Min1':6, 'Typ':7}
yesNo_mapping = {'Y':1, 'N':0}
paved_mapping = {'N':0, 'Y':2, 'P':2}
garagefinish_mapping = {'Fin': 3,'RFn': 2,'Unf': 1,'None': 0}
garage_qual_mapping = {'None':0, 'Grvl':1, 'Pave':2}
fence_mapping = {'None': 0,'MnWw': 1,'GdWo': 2,'MnPrv': 3, 'GdPrv': 4}
garage_cols = ['GarageType']
for col in garage_cols:
    df[col+'_encoded'] = df[col].map(garagefinish_mapping)
    df.drop(columns=[col], inplace=True)
functional_cols = ['Functional']
for col in functional_cols:
    df[col+'_encoded'] = df[col].map(functional_mapping)
    df.drop(columns=[col], inplace=True)
Yes_No_cols = ['CentralAir']
for col in Yes_No_cols:
    df[col+'_encoded'] = df[col].map(yesNo_mapping)
    df.drop(columns=[col], inplace=True)
quality2_cols = ['BsmtExposure', 'BsmtFinType1', ]
for col in quality2_cols:
    df[col+'_encoded'] = df[col].map(bsmyexposure_mapping)
    df.drop(columns=[col], inplace=True)
quality_cols = ['ExterQual', 'ExterCond','MasVnrType', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'GarageFinish', 'GarageCond', 'PoolQC']
for col in quality_cols:
    df[col+'_Encoded'] = df[col].map(quality_mapping)
    df.drop(columns=[col], inplace=True)
df['paved_encoded'] = df['PavedDrive'].map(paved_mapping)
df['BsmtCond_Encoded'] = df['BsmtCond'].map(bsmt_cond_mapping)
df['LotShape_Encoded'] = df['LotShape'].map(lotshape_mapping)
df['LandContour_Encoded'] = df['LandContour'].map(landContour_mapping)
df['LandSlope_encoded'] = df['LandSlope'].map(landslope_mapping)
df['GarageQual_encoded'] = df['GarageQual'].map(garage_qual_mapping)
df['Fence_encoded'] = df['Fence'].map(fence_mapping)
df = df.drop(columns = ['LotShape', 'LandContour', 'LandSlope', 'BsmtCond', 'PavedDrive'])

In [614]:
df.isna().sum()[20:30]

BsmtHalfBath    0
FullBath        0
HalfBath        0
BedroomAbvGr    0
KitchenAbvGr    0
TotRmsAbvGrd    0
Fireplaces      0
GarageYrBlt     0
GarageCars      0
GarageArea      0
dtype: int64

np.int64(0)